In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
import tensorflow as tf
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/train.parquet')

In [ ]:
df.columns

In [ ]:
# Add a small constant to prevent division by zero
epsilon = 1e-10

# --- Imbalances & Spreads ---
# Captures the immediate pressure on the order book
df['order_book_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['bid_qty'] + df['ask_qty'] + epsilon)

# Captures the pressure from market orders
df['trade_imbalance'] = (df['buy_qty'] - df['sell_qty']) / (df['buy_qty'] + df['sell_qty'] + epsilon)

# Simple difference in quoted quantities
df['quantity_spread'] = df['ask_qty'] - df['bid_qty']


# --- Sizes & Proxies ---
# Average size of trades in that minute
df['avg_trade_size'] = df['volume'] / (df['buy_qty'] + df['sell_qty'] + epsilon)

# Your WAP proxy - a measure of price pressure based on order book depth
df['wap_proxy'] = (df['bid_qty'] - df['ask_qty']) / (df['bid_qty'] + df['ask_qty'] + epsilon)


# --- Log-Transformed Features ---
# Reduces skewness in features with large values
df['log_volume'] = np.log1p(df['volume'])
df['log_ask_qty'] = np.log1p(df['ask_qty'])
df['log_bid_qty'] = np.log1p(df['bid_qty'])


# --- Interaction Feature ---
# Combines imbalance with total volume
df['imbalance_times_volume'] = df['order_book_imbalance'] * df['volume']

# Display the new columns to verify
print(df.columns)

In [ ]:
x_cols = [f'X{i}' for i in range(1, 781)]
df['X_mean'] = df[x_cols].mean(axis=1)
df['X_std'] = df[x_cols].std(axis=1)
df['X_skew'] = df[x_cols].skew(axis=1)
df = df.copy()

In [ ]:
df['X_n_positive'] = (df[x_cols] > 0).sum(axis=1)
df['X_n_negative'] = (df[x_cols] < 0).sum(axis=1)

In [ ]:
# Quantiles provide a robust summary of the distribution
df['X_median'] = df[x_cols].median(axis=1)
df['X_q25'] = df[x_cols].quantile(0.25, axis=1)
df['X_q75'] = df[x_cols].quantile(0.75, axis=1)

# Kurtosis measures the "tailedness" (presence of outliers)
df['X_kurtosis'] = df[x_cols].kurtosis(axis=1)

In [ ]:
# A small constant to prevent division by zero
epsilon = 1e-10

# What proportion of the minute's total volume was from aggressive buyers?
df['buy_proportion'] = df['buy_qty'] / (df['volume'] + epsilon)

# A simple measure of overall market depth/liquidity on the books
df['total_depth'] = df['bid_qty'] + df['ask_qty']

In [ ]:
# --- Polynomial Features ---

# Squaring key features can help capture non-linear effects
df['wap_proxy_sq'] = df['wap_proxy']**2
df['X_mean_sq'] = df['X_mean']**2

In [ ]:
epsilon = 1e-10

# --- Market Activity Intensity ---
# Measures how much of the standing liquidity on the book was consumed by trades.
# A high value might indicate a significant market-moving event.
df['activity_intensity'] = df['volume'] / (df['bid_qty'] + df['ask_qty'] + epsilon)

# --- Imbalance Delta ---
# The difference between executed trade pressure and quoted book pressure.
# A large value suggests aggressive actors are overwhelming the passive ones.
df['imbalance_delta'] = df['trade_imbalance'] - df['order_book_imbalance']

In [ ]:
# You should have already created 'trade_imbalance' and 'total_depth'
df['flow_depth_interaction'] = df['trade_imbalance'] * df['total_depth']

In [ ]:
# Define groups of X features
n_groups = 5
group_size = len(x_cols) // n_groups # 780 / 5 = 156

for i in range(n_groups):
    start_index = i * group_size
    end_index = start_index + group_size
    group_cols = x_cols[start_index:end_index]
    
    # Create mean and std for each group
    df[f'X_group_{i+1}_mean'] = df[group_cols].mean(axis=1)
    df[f'X_group_{i+1}_std'] = df[group_cols].std(axis=1)

In [ ]:
x_values = df[x_cols].values

# Calculate the number of times the sign of the signal changes
df['X_zero_crossings'] = np.sum(np.diff(np.sign(x_values), axis=1) != 0, axis=1)

In [ ]:
# Downcast integer columns
int_cols = df.select_dtypes(include=['int64']).columns
for col in int_cols:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# Downcast float columns
float_cols = df.select_dtypes(include=['float64']).columns
for col in float_cols:
    df[col] = pd.to_numeric(df[col], downcast='float')

print("Memory usage after downcasting:")
df.info(memory_usage='deep')

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'timestamp'}, inplace=True)
print(df.columns)
df.head()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Step 2: Define the validation period (February 2024, as planned)
validation_start_date = pd.to_datetime('2024-02-01')

# Step 3: Split the data into training and validation sets
train_df = df[df['timestamp'] < validation_start_date].copy()
val_df = df[df['timestamp'] >= validation_start_date].copy()

# Step 4: Separate features (X) from the target (y)
# Create a list of all feature columns to use
features = [col for col in train_df.columns if col not in ['label', 'timestamp']]

X_train = train_df[features]
y_train = train_df['label']

X_val = val_df[features]
y_val = val_df['label']

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

In [ ]:
# --- Step 1: Learn thresholds ONLY from the training data ---
high_volume_threshold = train_df['volume'].quantile(0.95)
high_x_std_threshold = train_df['X_std'].quantile(0.95)

print(f"High Volume Threshold (95th percentile): {high_volume_threshold}")
print(f"High X_std Threshold (95th percentile): {high_x_std_threshold}")

# --- Step 2: Create the new flag features on both datasets ---
# Use .copy() to avoid SettingWithCopyWarning
X_train = X_train.copy()
X_val = X_val.copy()

X_train['is_high_volume'] = (train_df['volume'] > high_volume_threshold).astype(int)
X_train['is_high_x_volatility'] = (train_df['X_std'] > high_x_std_threshold).astype(int)

X_val['is_high_volume'] = (val_df['volume'] > high_volume_threshold).astype(int)
X_val['is_high_x_volatility'] = (val_df['X_std'] > high_x_std_threshold).astype(int)


print("\nNew feature shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

In [ ]:
# Step 1: Initialize the scaler
scaler = StandardScaler()

# Step 2: Fit the scaler ONLY on the training feature data
scaler.fit(X_train)

# Step 3: Transform your training and validation sets
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

# The output of this step, X_train_scaled and X_val_scaled, are NumPy arrays.
# This is the format your model will expect.
print("Data scaling complete. Ready for model building.")

In [ ]:
# --- 1. Custom Loss Function to Maximize Pearson Correlation ---
# The goal is to minimize the negative Pearson correlation.
def pearson_loss(y_true, y_pred):
    y_true_mean = tf.reduce_mean(y_true)
    y_pred_mean = tf.reduce_mean(y_pred)
    y_true_centered = y_true - y_true_mean
    y_pred_centered = y_pred - y_pred_mean
    
    # Add a small epsilon to prevent division by zero
    corr = (tf.reduce_sum(y_true_centered * y_pred_centered) /
            (tf.sqrt(tf.reduce_sum(tf.square(y_true_centered))) *
             tf.sqrt(tf.reduce_sum(tf.square(y_pred_centered))) + 1e-7))
    
    return -corr # We minimize the negative correlation

In [ ]:
import lightgbm as lgb
from scipy.stats import pearsonr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Initialize and train the LightGBM Regressor ---
lgb_model = lgb.LGBMRegressor(
    objective='regression_l1', # L1 loss is robust for this kind of data
    n_estimators=2000,         # High number, but we'll use early stopping
    learning_rate=0.01,
    num_leaves=40,
    n_jobs=-1,                 # Use all available CPU cores
    seed=42,
    colsample_bytree=0.7,      # Use a random subset of features for each tree
    subsample=0.7              # Use a random subset of data for each tree
)

print("--- Training LightGBM model ---")
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='l1',
    callbacks=[lgb.early_stopping(100, verbose=True)] # Stop after 100 rounds of no improvement
)

# --- Evaluate the LightGBM model ---
lgb_predictions = lgb_model.predict(X_val)
correlation_lgb, _ = pearsonr(y_val, lgb_predictions)
print(f"\nLightGBM Pearson Correlation on Validation Set: {correlation_lgb:.6f}")

In [ ]:
# --- Get and Plot Feature Importance ---
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
}).sort_values(by='importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=importance_df.head(30))
plt.title('Top 30 Most Important Features from LightGBM')
plt.show()

# --- Select the Top 200 Features ---
N_TOP_FEATURES = 200
top_features = importance_df.head(N_TOP_FEATURES)['feature'].tolist()

# Create new DataFrames with only the top features
X_train_top = X_train[top_features]
X_val_top = X_val[top_features]

# We need to re-scale the data now that we have a different set of columns
scaler_top = StandardScaler()
X_train_top_scaled = scaler_top.fit_transform(X_train_top)
X_val_top_scaled = scaler_top.transform(X_val_top)

print(f"\nSelected top {N_TOP_FEATURES} features. New training shape: {X_train_top_scaled.shape}")

In [ ]:
# --- Define a simpler MLP architecture ---
n_features_top = X_train_top_scaled.shape[1]

mlp_model = Sequential([
    InputLayer(input_shape=(n_features_top,)),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='linear')
])

optimizer = Adam(learning_rate=0.0005)
mlp_model.compile(optimizer=optimizer, loss=pearson_loss)

# --- Define Callbacks and Train ---
checkpoint_mlp = ModelCheckpoint('best_mlp_model.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_mlp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping_mlp = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

print("\n--- Training Refined MLP model on Top Features ---")
history_mlp = mlp_model.fit(
    X_train_top_scaled, y_train,
    validation_data=(X_val_top_scaled, y_val),
    epochs=100,
    batch_size=1024,
    callbacks=[checkpoint_mlp, reduce_lr_mlp, early_stopping_mlp],
    verbose=1
)

# --- Evaluate the refined MLP model ---
mlp_predictions = mlp_model.predict(X_val_top_scaled).flatten()
correlation_mlp, _ = pearsonr(y_val, mlp_predictions)
print(f"\nRefined MLP Pearson Correlation on Validation Set: {correlation_mlp:.6f}")

In [ ]:
# --- Create the Ensemble Prediction ---
# We already have lgb_predictions and mlp_predictions
ensemble_predictions = (lgb_predictions * 0.5) + (mlp_predictions * 0.5)

# --- Evaluate the Ensemble ---
correlation_ensemble, _ = pearsonr(y_val, ensemble_predictions)
print(f"\nEnsemble Pearson Correlation on Validation Set: {correlation_ensemble:.6f}")

In [ ]:
import optuna
from scipy.stats import pearsonr
import lightgbm as lgb

def objective(trial):
    """Define the objective function for Optuna to optimize."""
    
    # Define the search space for hyperparameters
    params = {
        'objective': 'regression_l1',
        'metric': 'l1',
        'n_estimators': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'n_jobs': -1,
        'seed': 42,
        'boosting_type': 'gbdt',
    }
    
    # Train the model with the suggested params
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='l1',
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    
    # Make predictions and calculate Pearson correlation
    preds = model.predict(X_val)
    # We want to maximize correlation, so Optuna will maximize this value
    pearson_corr, _ = pearsonr(y_val, preds)
    
    return pearson_corr

# --- Start the Optuna study ---
# We want to 'maximize' the Pearson correlation
study = optuna.create_study(direction='maximize')
# Run 30 trials to find the best params. You can increase this for a more thorough search.
study.trials_dataframe().to_csv('tuner_results.csv')
study.optimize(objective, n_trials=30)

# --- Print the best results ---
print("\nBest trial:")
trial = study.best_trial
print(f"  Value (Pearson Correlation): {trial.value:.6f}")
print("  Best hyperparameters: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# --- Train the final best model with the optimal parameters ---
best_params = trial.params
final_lgb_model = lgb.LGBMRegressor(n_estimators=2000, **best_params)
final_lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='l1',
    callbacks=[lgb.early_stopping(100)]
)

In [ ]:
# --- Get predictions from your best models ---
tuned_lgb_preds = final_lgb_model.predict(X_val)
# mlp_predictions should still be stored from your previous run

# --- Create a weighted ensemble ---
# Give more weight to the stronger model (LGBM)
weighted_ensemble_preds = (tuned_lgb_preds * 0.7) + (mlp_predictions * 0.3)

# --- Evaluate the new ensemble ---
correlation_weighted_ensemble, _ = pearsonr(y_val, weighted_ensemble_preds)
print(f"\nWeighted Ensemble Pearson Correlation: {correlation_weighted_ensemble:.6f}")

In [ ]:
import lightgbm as lgb
from scipy.stats import pearsonr

# --- Get the best hyperparameters from your completed study ---
best_params = study.best_trial.params

# --- Train the final model with a fixed number of iterations ---
# We are removing the early stopping callback for this final fit.
print("Training the final, tuned LightGBM model...")
final_lgb_model = lgb.LGBMRegressor(
    n_estimators=400, # Train for a fixed 400 rounds
    **best_params     # Use the best params found by Optuna
)

# Fit the model on the full training data
final_lgb_model.fit(X_train, y_train)

# --- Get predictions from the PROPERLY trained model ---
tuned_lgb_preds = final_lgb_model.predict(X_val)
correlation_tuned_lgb, _ = pearsonr(y_val, tuned_lgb_preds)
print(f"\nCorrectly Trained LightGBM Correlation: {correlation_tuned_lgb:.6f}")

In [ ]:
# mlp_predictions should still be stored from your previous MLP run

# --- Create the new weighted ensemble ---
weighted_ensemble_preds = (tuned_lgb_preds * 0.7) + (mlp_predictions * 0.3)

# --- Evaluate the new ensemble ---
correlation_weighted_ensemble, _ = pearsonr(y_val, weighted_ensemble_preds)
print(f"Final Weighted Ensemble Pearson Correlation: {correlation_weighted_ensemble:.6f}")

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# We'll use your original training dataframe before splitting
# Ensure it contains all original and engineered features
# Let's call it `full_train_df` for this example. Assuming you have `train_df` ready.

print("Calculating correlation of all features with the label...")
# This computes the Pearson correlation of every column with 'label'
# .abs() gets the absolute correlation, as a strong negative one is also good
correlations = train_df.corr(method='pearson')['label'].abs().sort_values(ascending=False)

print("\n--- Top 20 Features Most Correlated with 'label' ---")
print(correlations.head(20))

# --- Test the single best feature as a prediction ---
# The top feature will be 'label' itself (corr=1.0), so we take the second one
golden_feature_name = correlations.index[1]
print(f"\nThe most likely 'Golden Feature' is: {golden_feature_name}")

# Let's see what score we get by using only this feature's values as our prediction
# We will use the validation set for this test
golden_feature_predictions = val_df[golden_feature_name]
correlation_golden, _ = pearsonr(y_val, golden_feature_predictions)

print(f"\nCorrelation score using ONLY '{golden_feature_name}' as prediction: {correlation_golden:.6f}")

In [ ]:
test_df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/test.parquet')
test_df.head()

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load the test data ---
print("Loading test data...")
test_df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/test.parquet')
test_ids = test_df['ID']


# --- 2. Apply ALL Feature Engineering Steps ---
print("Applying feature engineering to the test set...")
epsilon = 1e-10

# Imbalances, Spreads, Sizes, Proxies, Logs, and basic interactions
test_df['order_book_imbalance'] = (test_df['bid_qty'] - test_df['ask_qty']) / (test_df['bid_qty'] + test_df['ask_qty'] + epsilon)
test_df['trade_imbalance'] = (test_df['buy_qty'] - test_df['sell_qty']) / (test_df['buy_qty'] + test_df['sell_qty'] + epsilon)
test_df['quantity_spread'] = test_df['ask_qty'] - test_df['bid_qty']
test_df['avg_trade_size'] = test_df['volume'] / (test_df['buy_qty'] + test_df['sell_qty'] + epsilon)
test_df['wap_proxy'] = (test_df['bid_qty'] - test_df['ask_qty']) / (test_df['bid_qty'] + test_df['ask_qty'] + epsilon)
test_df['log_volume'] = np.log1p(test_df['volume'])
test_df['log_ask_qty'] = np.log1p(test_df['ask_qty'])
test_df['log_bid_qty'] = np.log1p(test_df['bid_qty'])
test_df['imbalance_times_volume'] = test_df['order_book_imbalance'] * test_df['volume']

# X-feature statistics
test_df['X_mean'] = test_df[x_cols].mean(axis=1)
test_df['X_std'] = test_df[x_cols].std(axis=1)
test_df['X_skew'] = test_df[x_cols].skew(axis=1)
test_df['X_n_positive'] = (test_df[x_cols] > 0).sum(axis=1)
test_df['X_n_negative'] = (test_df[x_cols] < 0).sum(axis=1)
test_df['X_median'] = test_df[x_cols].median(axis=1)
test_df['X_q25'] = test_df[x_cols].quantile(0.25, axis=1)
test_df['X_q75'] = test_df[x_cols].quantile(0.75, axis=1)
test_df['X_kurtosis'] = test_df[x_cols].kurtosis(axis=1)

# Advanced ratios, interactions, and polynomial features
test_df['buy_proportion'] = test_df['buy_qty'] / (test_df['volume'] + epsilon)
test_df['total_depth'] = test_df['bid_qty'] + test_df['ask_qty']
test_df['wap_proxy_sq'] = test_df['wap_proxy']**2
test_df['X_mean_sq'] = test_df['X_mean']**2
test_df['activity_intensity'] = test_df['volume'] / (test_df['bid_qty'] + test_df['ask_qty'] + epsilon)
test_df['imbalance_delta'] = test_df['trade_imbalance'] - test_df['order_book_imbalance']
test_df['flow_depth_interaction'] = test_df['trade_imbalance'] * test_df['total_depth']

# X-group and X-sequence features
n_groups = 5
group_size = len(x_cols) // n_groups
for i in range(n_groups):
    start_index = i * group_size
    end_index = start_index + group_size
    group_cols = x_cols[start_index:end_index]
    test_df[f'X_group_{i+1}_mean'] = test_df[group_cols].mean(axis=1)
    test_df[f'X_group_{i+1}_std'] = test_df[group_cols].std(axis=1)
test_df['X_zero_crossings'] = np.sum(np.diff(np.sign(test_df[x_cols].values), axis=1) != 0, axis=1)


# --- 3. Predict with Tuned LightGBM ---
print("Predicting with LightGBM...")
# Ensure test_df columns are in the same order as X_train for the model
lgb_test_preds = final_lgb_model.predict(test_df[X_train.columns])


# --- 4. Predict with Refined MLP ---
print("Predicting with MLP...")
# Select the same top features for the MLP
X_test_top = test_df[top_features]
# Use the scaler that was FITTED ON THE TRAINING DATA to transform the test data
X_test_top_scaled = scaler_top.transform(X_test_top)
# The mlp_model is the one trained on top features
mlp_test_preds = mlp_model.predict(X_test_top_scaled).flatten()


# --- 5. Create the Final Weighted Ensemble ---
print("Creating weighted ensemble...")
final_test_predictions = (lgb_test_preds * 0.7) + (mlp_test_preds * 0.3)


# --- 6. Generate the submission file ---
print("Creating submission file...")
submission_df = pd.DataFrame({'ID': test_ids, 'label': final_test_predictions})
submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' has been created successfully!")